In [1]:
import numpy as np
import gensim
from gensim.models import Word2Vec,KeyedVectors

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

e:\my_files\anaconda3\envs\pabase\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


<h3>Load Saved Model</h3>

In [2]:
FOLDER_PATH = "E:/TEMP/GGL_W2V"

# Load Google's pre-trained Word2Vec model.
model = KeyedVectors.load_word2vec_format(FOLDER_PATH+'/GoogleNews-vectors-negative300.bin', binary=True)  

In [3]:
print("Vocabulary Size: {0}".format(len(model.vocab)))
#3,000,000

Vocabulary Size: 3000000


In [4]:
VOCAB_SIZE = len(model.vocab)-1
EMBEDDING_DIM = model["is"].shape[0]#model.trainables.layer1_size

w2v = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

<h3>Prepare Vocab File</h3>

In [5]:
#save .tsv with vocabulary
tsv_file_path = FOLDER_PATH+"/tensorboard/metadata.tsv"
with open(tsv_file_path,'w+', encoding='utf-8') as file_metadata:
    for i,word in enumerate(model.index2word[:VOCAB_SIZE]):
        w2v[i] = model[word]
        file_metadata.write(word+'\n')

<h3>Tensorflow Projector</h3>

In [6]:
TENSORBOARD_FILES_PATH = FOLDER_PATH+"/tensorboard"

#Tensorflow Graph Variables
X_init = tf.placeholder(tf.float32, shape=(VOCAB_SIZE, EMBEDDING_DIM), name="embedding")
X = tf.Variable(X_init)

#Initialize Variables
init = tf.global_variables_initializer()

sess = tf.Session()
#sess.run(tf.initialize_all_variables(), feed_dict={X_init: w2v})
sess.run(init, feed_dict={X_init: w2v})

saver = tf.train.Saver()
writer = tf.summary.FileWriter(TENSORBOARD_FILES_PATH, sess.graph)

#projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.metadata_path = tsv_file_path

In [7]:
projector.visualize_embeddings(writer,config)
saver.save(sess, TENSORBOARD_FILES_PATH+'/model.ckpt', global_step = VOCAB_SIZE)

#run python -m tensorboard.main --logdir=C:\....

'E:/TEMP/GGL_W2V/tensorboard/model.ckpt-2999999'

In [8]:
sess.close()

In [ ]:
w2v.shape

<h3>Scrap Code</h3>

In [ ]:
TENSORBOARD_FILES_PATH = FOLDER_PATH+"/tensorboard"

sess = tf.Session()

X_init = tf.placeholder(tf.float32, shape=(VOCAB_SIZE, EMBEDDING_DIM))
X = tf.Variable(X_init)
# The rest of the setup...
sess.run(tf.initialize_all_variables(), feed_dict={X_init: w2v})

In [ ]:
saver = tf.train.Saver()
writer = tf.summary.FileWriter(TENSORBOARD_FILES_PATH, sess.graph)

#projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.metadata_path = tsv_file_path

In [ ]:
projector.visualize_embeddings(writer,config)
saver.save(sess, TENSORBOARD_FILES_PATH+'/model.ckpt', global_step = VOCAB_SIZE)

In [ ]:
sess.close()

In [ ]:
TENSORBOARD_FILES_PATH = FOLDER_PATH+"\tensorboard"
sess = tf.InteractiveSession()

with tf.device("/cpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name = "embedding")

tf.global_variables_initializer().run()
saver = tf.train.Saver()
writer = tf.summary.FileWriter(TENSORBOARD_FILES_PATH, sess.graph)

#projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.metadata_path = tsv_file_path